In [1]:
import re
import numpy as np
import collections
from sklearn import metrics
from sklearn.cross_validation import train_test_split
import tensorflow as tf
import pandas as pd
from unidecode import unidecode
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
import time

/usr/local/lib/python3.6/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
import json

with open('tokenization.json') as fopen:
    dataset = json.load(fopen)
texts = dataset['texts']
labels = dataset['labels']
del dataset

In [3]:
x, y = [], []
for i in tqdm(range(len(texts))):
    s = ' '.join(texts[i])
    if len(s) > 5:
        x.append(s)
        y.append(labels[i])

100%|██████████| 675023/675023 [00:00<00:00, 1247938.47it/s]


In [4]:
def convert_sparse_matrix_to_sparse_tensor(X):
    coo = X.tocoo()
    indices = np.mat([coo.row, coo.col]).transpose()
    return tf.SparseTensorValue(indices, coo.col, coo.shape), tf.SparseTensorValue(indices, coo.data, coo.shape)

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

In [6]:
bow_chars = CountVectorizer(ngram_range=(3, 5), analyzer='char_wb', max_features=300000).fit(x)
delattr(bow_chars, 'stop_words_')

In [7]:
vectors = bow_chars.transform(x)
feature_shape = vectors.shape[1]
feature_shape

300000

In [8]:
class Model:
    def __init__(self, output_size, vocab_size, learning_rate):
        self.X = tf.sparse_placeholder(tf.int32)
        self.W = tf.sparse_placeholder(tf.int32)
        self.Y = tf.placeholder(tf.int32, [None])
        embeddings = tf.Variable(tf.truncated_normal([vocab_size,128]))
        embed = tf.nn.embedding_lookup_sparse(embeddings, self.X, self.W, combiner='mean')
        self.logits = tf.layers.dense(embed, output_size)
        self.cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits = self.logits, labels = self.Y))
        self.optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(self.cost)
        correct_pred = tf.equal(tf.argmax(self.logits, 1,output_type=tf.int32), self.Y)
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [9]:
sess = tf.InteractiveSession()
model = Model(2, feature_shape, 1e-4)
sess.run(tf.global_variables_initializer())

/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [10]:
saver = tf.train.Saver(tf.trainable_variables())
saver.save(sess, 'fast-text-char/model.ckpt')

'fast-text-char/model.ckpt'

In [11]:
train_X, test_X, train_Y, test_Y = train_test_split(
    vectors, y, test_size = 0.2
)

In [12]:
from tqdm import tqdm
import time

batch_size = 60
EARLY_STOPPING, CURRENT_CHECKPOINT, CURRENT_ACC, EPOCH = 3, 0, 0, 0

while True:
    lasttime = time.time()
    if CURRENT_CHECKPOINT == EARLY_STOPPING:
        print('break epoch:%d\n' % (EPOCH))
        break

    train_acc, train_loss, test_acc, test_loss = 0, 0, 0, 0
    pbar = tqdm(
        range(0, train_X.shape[0], batch_size), desc = 'train minibatch loop'
    )
    for i in pbar:
        batch_x = convert_sparse_matrix_to_sparse_tensor(train_X[i : min(i + batch_size, train_X.shape[0])])
        batch_y = train_Y[i : min(i + batch_size, train_X.shape[0])]
        acc, cost, _ = sess.run(
            [model.accuracy, model.cost, model.optimizer],
            feed_dict = {
                model.Y: batch_y,
                model.X: batch_x[0],
                model.W: batch_x[1],
            },
        )
        assert not np.isnan(cost)
        train_loss += cost
        train_acc += acc
        pbar.set_postfix(cost = cost, accuracy = acc)

    pbar = tqdm(range(0, test_X.shape[0], batch_size), desc = 'test minibatch loop')
    for i in pbar:
        batch_x = convert_sparse_matrix_to_sparse_tensor(test_X[i : min(i + batch_size, test_X.shape[0])])
        batch_y = test_Y[i : min(i + batch_size, test_X.shape[0])]
        acc, cost = sess.run(
            [model.accuracy, model.cost],
            feed_dict = {
                model.Y: batch_y,
                model.X: batch_x[0],
                model.W: batch_x[1],
            },
        )
        test_loss += cost
        test_acc += acc
        pbar.set_postfix(cost = cost, accuracy = acc)

    train_loss /= train_X.shape[0] / batch_size
    train_acc /= train_X.shape[0] / batch_size
    test_loss /= test_X.shape[0] / batch_size
    test_acc /= test_X.shape[0] / batch_size

    if test_acc > CURRENT_ACC:
        print(
            'epoch: %d, pass acc: %f, current acc: %f'
            % (EPOCH, CURRENT_ACC, test_acc)
        )
        CURRENT_ACC = test_acc
        CURRENT_CHECKPOINT = 0
    else:
        CURRENT_CHECKPOINT += 1

    print('time taken:', time.time() - lasttime)
    print(
        'epoch: %d, training loss: %f, training acc: %f, valid loss: %f, valid acc: %f\n'
        % (EPOCH, train_loss, train_acc, test_loss, test_acc)
    )
    EPOCH += 1

train minibatch loop:   0%|          | 3/8935 [00:00<06:10, 24.11it/s, accuracy=0.817, cost=0.422]

epoch: 0, pass acc: 0.000000, current acc: 0.753968
time taken: 375.06318616867065
epoch: 0, training loss: 0.557531, training acc: 0.718055, valid loss: 0.512189, valid acc: 0.753968



train minibatch loop:   0%|          | 3/8935 [00:00<06:09, 24.17it/s, accuracy=0.767, cost=0.41] 

epoch: 1, pass acc: 0.753968, current acc: 0.766470
time taken: 375.01256942749023
epoch: 1, training loss: 0.498124, training acc: 0.763182, valid loss: 0.494869, valid acc: 0.766470



train minibatch loop:   0%|          | 3/8935 [00:00<06:07, 24.33it/s, accuracy=0.8, cost=0.404]  

...

time taken: 374.5051779747009
epoch: 9, training loss: 0.460240, training acc: 0.788499, valid loss: 0.481923, valid acc: 0.773894



train minibatch loop:   0%|          | 3/8935 [00:00<06:07, 24.29it/s, accuracy=0.817, cost=0.374]

time taken: 374.56684923171997
epoch: 10, training loss: 0.458494, training acc: 0.789598, valid loss: 0.482133, valid acc: 0.773996



test minibatch loop: 100%|██████████| 2234/2234 [00:06<00:00, 358.37it/s, accuracy=0.69, cost=0.535] 

time taken: 374.5287504196167
epoch: 11, training loss: 0.456897, training acc: 0.790584, valid loss: 0.482434, valid acc: 0.773913

break epoch:12



In [13]:
real_Y, predict_Y = [], []

pbar = tqdm(
    range(0, test_X.shape[0], batch_size), desc = 'validation minibatch loop'
)
for i in pbar:
    batch_x = convert_sparse_matrix_to_sparse_tensor(test_X[i : min(i + batch_size, test_X.shape[0])])
    batch_y = test_Y[i : min(i + batch_size, test_X.shape[0])]
    predict_Y += np.argmax(
        sess.run(
            model.logits, feed_dict = {model.X: batch_x[0], model.W: batch_x[1], model.Y: batch_y}
        ),
        1,
    ).tolist()
    real_Y += batch_y

validation minibatch loop: 100%|██████████| 2234/2234 [00:03<00:00, 607.53it/s]


In [14]:
print(
    metrics.classification_report(
        real_Y, predict_Y, target_names = ['negative','positive']
    )
)

             precision    recall  f1-score   support

   negative       0.78      0.80      0.79     70256
   positive       0.77      0.75      0.76     63766

avg / total       0.77      0.77      0.77    134022



In [16]:
saver.save(sess, 'fast-text-char/model.ckpt')

'fast-text-char/model.ckpt'

In [17]:
import pickle
with open('vectorizer-sparse-sentiment.pkl','wb') as fopen:
    pickle.dump(bow_chars, fopen)